In [10]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import math
df = pd.read_csv('../data/data2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         2112 non-null   int64  
 1   Substrate  2112 non-null   object 
 2   Ceramic    2112 non-null   object 
 3   Thickness  2112 non-null   float64
 4   Lsub       2112 non-null   float64
 5   asub       2112 non-null   float64
 6   bsub       2112 non-null   float64
 7   Lcer       2112 non-null   float64
 8   acer       2112 non-null   float64
 9   bcer       2112 non-null   float64
 10  L          2112 non-null   float64
 11  a          2112 non-null   float64
 12  b          2112 non-null   float64
dtypes: float64(10), int64(1), object(2)
memory usage: 214.6+ KB


In [11]:
###Predict L
X, y = df[['Thickness','Lsub', 'asub','bsub', 'Lcer', 'acer', 'bcer']], df[['L', 'a', 'b']]

#Perform Scaling if needed
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_sc = sc.fit_transform(X)

#Splitting the data into train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [12]:
# create dataset
X, y = pd.DataFrame(df[['Thickness', 'Lsub', 'asub','bsub', 'Lcer', 'acer', 'bcer']]).to_numpy(), pd.DataFrame(df[['L','a','b']]).to_numpy()
# summarize shape
print(X_train.shape, y_train.shape)

(1689, 7) (1689, 3)


In [13]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

# get the dataset
def get_dataset():
    
    X, y = pd.DataFrame(df[['Thickness', 'Lsub', 'asub','bsub', 'Lcer', 'acer', 'bcer']]).to_numpy(), pd.DataFrame(df[['L','a','b']]).to_numpy()
    return X, y

# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		print('>%.3f' % mae)
		results.append(mae)
	return results

# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

>1.401
>1.702
>1.644
>1.617
>1.392
>2.056
>1.326
>1.516
>1.344
>1.907
>1.314
>1.764
>1.551
>1.366
>1.726
MAE: 1.575 (0.219)


In [14]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [15]:
def createmodel(n_layers, first_layer_nodes, last_layer_nodes, activation_func, loss_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(3, activation='linear'))
    model.compile(optimizer='adam', loss=loss_func, metrics = ["accuracy"]) #note: metrics could also be 'mse'
    
    return model

##Wrap model into scikit-learn
model =  KerasRegressor(build_fn=createmodel, verbose = False)  

In [16]:
activation_funcs = ['relu']#['sigmoid', 'relu', 'tanh'] 
loss_funcs = ['mean_squared_error'] #['mean_squared_error','hinge','mae']
param_grid = dict(n_layers=[1,2], first_layer_nodes = [7, 5], last_layer_nodes = [5, 3],  activation_func = activation_funcs, loss_func = loss_funcs, batch_size = [50], epochs = [10])
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv=10, n_jobs=-1, verbose=1, scoring = "neg_mean_squared_error")

In [17]:
%%time
grid.fit(X_train,y_train)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


exception calling callback for <Future at 0x14dd56213d0 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\groch\.conda\envs\GPU\lib\site-packages\joblib\externals\loky\process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\groch\.conda\envs\GPU\lib\multiprocessing\queues.py", line 117, in get
    res = self._recv_bytes()
  File "C:\Users\groch\.conda\envs\GPU\lib\multiprocessing\connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "C:\Users\groch\.conda\envs\GPU\lib\multiprocessing\connection.py", line 323, in _recv_bytes
    return self._get_more_data(ov, maxsize)
  File "C:\Users\groch\.conda\envs\GPU\lib\multiprocessing\connection.py", line 349, in _get_more_data
    f.write(ov.getbuffer())
MemoryError
"""

The above exception was the direct cause of the following exception:



In [18]:
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
score_df = pd.DataFrame(grid.cv_results_)
score_df.sort_values(by='rank_test_score').head(5)

In [25]:
pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

,mean_test_score,std_test_score,params
0,NaN,NaN,"{'activation_func': 'relu', 'batch_size': 50, ..."
1,-24.490260,14.922815,"{'activation_func': 'relu', 'batch_size': 50, ..."
2,NaN,NaN,"{'activation_func': 'relu', 'batch_size': 50, ..."
3,-25.036329,11.882321,"{'activation_func': 'relu', 'batch_size': 50, ..."
4,NaN,NaN,"{'activation_func': 'relu', 'batch_size': 50, ..."
5,-23.875925,11.804245,"{'activation_func': 'relu', 'batch_size': 50, ..."
6,NaN,NaN,"{'activation_func': 'relu', 'batch_size': 50, ..."
7,-24.278635,9.413140,"{'activation_func': 'relu', 'batch_size': 50, ..."
8,NaN,NaN,"{'activation_func': 'relu', 'batch_size': 50, ..."
9,-51.353558,10.550953,"{'activation_func': 'relu', 'batch_size': 50, ..."


In [ ]:
# Function to generate Deep ANN model 
def make_regression_ann(Optimizer_trial):
    from keras.models import Sequential
    from keras.layers import Dense
    
    model = Sequential()
    model.add(Dense(units=10, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(3, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model
 
###########################################
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
 
# Listing all the parameters to try
Parameter_Trials={'batch_size':[10,20,30],
                      'epochs':[10,20],
                    'Optimizer_trial':['adam', 'rmsprop']
                 }
 
# Creating the regression ANN model
RegModel=KerasRegressor(make_regression_ann, verbose=0)
 
###########################################
from sklearn.metrics import make_scorer
 
# Defining a custom function to calculate accuracy
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)
 
custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)
 
#########################################
# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring='r2', 
                         cv=5)
 
#########################################
# Measuring how much time it took to find the best params
import time
StartTime=time.time()
 
# Running Grid Search for different paramenters
grid_search.fit(X,y, verbose=1)
 
EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')
 
print('### Printing Best parameters ###')
grid_search.best_params_

In [ ]:
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 10, epochs = 20, verbose=0)
 

Test_Data=PredictorScalerFit.inverse_transform(X_test)
 
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['L']=y_test_orig
TestingData['PredictedL']=Predictions
TestingData.head()

In [ ]:
# Computing the absolute percent error
APE=100*(abs(TestingData['L']-TestingData['PredictedL'])/TestingData['L'])
TestingData['APE']=APE
 
print('The Accuracy of ANN model is:', 100-np.mean(APE))
TestingData.head()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(TestingData['L'], TestingData['PredictedL'], edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], "k--", lw=4)
ax.set_xlabel("Measured")
ax.set_ylabel("Predicted")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(TestingData['L'], TestingData['APE'], edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], "k--", lw=4)
ax.set_xlabel("Measured")
ax.set_ylabel("APE")
plt.show()

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasRegressor
from keras import optimizers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Split the data into test and training sets.

x_label = ['Lsub','asub', 'bsub', 'Lcer','acer','bcer', 'Thickness']
y_label = ['L']
np.random.seed(100)
X_train, X_test, y_train, y_test = train_test_split(df[x_label],df[y_label],test_size=0.2)
# Print the dimensions
print('Training set dimensions X, y: ' + str(X_train.shape) + ' ' +str(y_train.shape))
print('Test set dimensions X, y: ' + str(X_test.shape) + ' '+ str(y_test.shape))

In [ ]:
# Define regression model in Keras
def regression_model():
    # Define model
    model = Sequential()
    model.add(Dense(units=10, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(3, kernel_initializer='normal'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['accuracy'])
    
    return model

# Use KerasRegressor wrapper (from Keras to sklearn)
# The packages we use are meant to be run with sklearn models
estimator = KerasRegressor(build_fn=regression_model, validation_split = 0.2, batch_size=100, epochs=100, verbose=0)
history = estimator.fit(X_train, y_train)

In [ ]:
# summarize history loss
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(estimator, random_state=1).fit(X_train,y_train)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

In [ ]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

# get the dataset
def get_dataset():
    x_label = ['Lsub','asub', 'bsub', 'Lcer','acer','bcer', 'Thickness']
    y_label = ['L','a','b']
    X, y = df[x_label], df[y_label]
    return X, y
# summarize shape
print(X.shape, y.shape)

In [ ]:
# Split the data into test and training sets.

x_label = ['Lsub','asub', 'bsub', 'Lcer','acer','bcer', 'Thickness']
y_label = ['L','a','b']
X_train, X_test, y_train, y_test = train_test_split(df[x_label],df[y_label],test_size=0.2)
# Print the dimensions
print('Training set dimensions X, y: ' + str(X_train.shape) + ' ' +str(y_train.shape))
print('Test set dimensions X, y: ' + str(X_test.shape) + ' '+ str(y_test.shape))

In [ ]:
# Define regression model in Keras
def regression_model():
    # Define model
    model = Sequential()
    model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    adam = optimizers.Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=adam,metrics=['accuracy'])
    return model

# Use KerasRegressor wrapper (from Keras to sklearn)
# The packages we use are meant to be run with sklearn models
estimator = KerasRegressor(build_fn=regression_model, validation_split = 0.2, batch_size=100, epochs=100, verbose=0)
history = estimator.fit(X_train, y_train)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(estimator, random_state=1).fit(X_train,y_train)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

In [ ]:
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
import shap

# SHAP expects model functions to take a 2D numpy array as input, so we define a wrapper function around the original Keras predict function.
def f_wrapper(X):
    return estimator.predict(X).flatten()

# Too many input data - use a random slice
# rather than use the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
X_train_summary = shap.kmeans(X_train, 20)

# Compute Shap values
explainer = shap.KernelExplainer(f_wrapper,X_train_summary)

# Make plot with combined shap values
# The training set is too big so let's sample it. We get enough point to draw conclusions
X_train_sample = X_train.sample(400)
shap_values  = explainer.shap_values(X_train_sample)
shap.summary_plot(shap_values, X_train_sample)

In [ ]:
# load dataset
X, y = get_dataset()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)
# make a prediction for new data
row = []
newX = asarray([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])

In [ ]:
# create dataset
X, y = df[['Thickness','Lsub', 'asub','bsub', 'Lcer', 'acer', 'bcer']].to_numpy(), df[['L', 'a', 'b']].to_numpy()
# summarize shape
print(X.shape, y.shape)

In [ ]:
# define the model
model = Sequential()
model.add(Dense(units=10, input_dim=7, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=15, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer='normal'))
model.compile(loss='mae', optimizer='adam')

In [ ]:
#get model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(units=10, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(3, kernel_initializer='normal'))
    model.compile(loss='mae', optimizer='adam')
    return model

In [ ]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = []
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=10, epochs=15)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
        results.append(mae)
    return results

In [ ]:
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

In [ ]:
# make a prediction for new data
row = [0.3,70,2.2,9,71, 0.2, 10]
newX = np.array([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])

In [6]:
import numpy as np
Lf = 10
Lcer = 7
Lsub = 20
np.clip(Lf, np.min([Lcer,Lsub]), np.max([Lcer,Lsub]))

10

In [4]:
np.max([10,20])

20